# Multinomial Naive Bayes on PyTorch

Below is a very simple demonstration of a multinomial Naive Bayes using PyTorch. It's very simple and doesn't include any input checking or advanced features. When the algorithm is moved over the cuML, it will also require that we have a `LabelBinarizer` on GPU. The Naive Bayes MNMG version will require that we have an MNMG version of `LabelBinarizer`. 


### A note on MNMG design
Since we only need the marginals of the classes for the prior and
the marginals of each term / feature, this algorithm is extremely
simple to parallelize over a cluster of workers. 

Assuming the set of terms is small enough to fit on each GPU:

1. The labels will first need to be one-hot encoded, which would require
taking the global union of the set of labels across the cluster, 
then the workers can binarize individually. 

2. After one-hot encoding the labels, the workers can compute the 
frequencies for the classes and the features. 

3. The frequencies are reduced on a single worker, who follows the 
remaining steps from the single GPU `train()`

4. For inference, the model is propagated out to all the workers with
inference partitions for embarrassingly parallel inference. 


In [218]:
import cupy

from sklearn.datasets import fetch_20newsgroups
from sklearn.preprocessing import LabelBinarizer

In [219]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [220]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [221]:
from sklearn.feature_extraction.text import CountVectorizer

In [222]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)

In [223]:
X_train_counts

<2257x35788 sparse matrix of type '<class 'numpy.int64'>'
	with 365886 stored elements in Compressed Sparse Row format>

In [224]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, twenty_train.target)

In [225]:
clf

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [226]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)

predicted = clf.predict(X_new_counts)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


In [227]:
X_train_counts.shape

(2257, 35788)

In [228]:
import torch
import numpy as np

def scipy_to_torch(sp):
    coo = sp.tocoo()
    values = coo.data
    indices = np.vstack((coo.row, coo.col))
    
    i = torch.cuda.LongTensor(indices)
    v = torch.cuda.FloatTensor(values)

    return torch.cuda.sparse.FloatTensor(i, v, torch.Size(coo.shape))

In [229]:
a = scipy_to_torch(X_train_counts)

In [230]:
import math

class MultinomialNB(object):
    
    def __init__(self, alpha=1.0, fit_prior=True, class_prior=None):
        self.alpha = alpha
        self.fit_prior = fit_prior
        self.class_prior = class_prior
        
        self.n_features_ = None
        
    def fit(self, X, y, _partial=False, _classes=None):

        self.n_features_ = X.shape[1]
        self._init_counters(y.shape[1], X.shape[1])

        self.classes_ = l.classes_
        self.n_classes_ = len(l.classes_)

        self._count(X, y)
        self._update_feature_log_prob(self.alpha)
        self._update_class_log_prior(class_prior=self.class_prior)
        
        return self
        
    
    def predict(self, X):
        jll = self._joint_log_likelihood(X)
        
        _, indices = torch.max(jll, 1)
        return indices
        
    def _init_counters(self, n_effective_classes, n_features):
        self.class_count_ = torch.zeros(n_effective_classes).cuda()
        self.feature_count_ = torch.zeros(n_effective_classes, n_features).cuda()
        

    def _count(self, X, Y):
        self.feature_count_ += torch.sparse.mm(X.t(), Y).t()
        self.class_count_ += Y.sum(axis=0)
        
    def _update_class_log_prior(self, class_prior=None):

        if class_prior is not None:
            self.class_log_prior_ = torch.log(class_prior)
            
        elif self.fit_prior:
            log_class_count = torch.log(self.class_count_)

        self.class_log_prior_ = torch.full((self.n_classes_,1), 
                                           -math.log(self.n_classes_)).cuda()
        
    def _update_feature_log_prob(self, alpha):
        """ apply add-lambda smoothing to raw counts and recompute log probabilities"""
        smoothed_fc = self.feature_count_ + alpha
        smoothed_cc = smoothed_fc.sum(axis=1).reshape(-1, 1)
        self.feature_log_prob_ = (torch.log(smoothed_fc) - torch.log(smoothed_cc))
        
    def _joint_log_likelihood(self, X):
        """ Calculate the posterior log probability of the samples X """
        ret = torch.sparse.mm(X, self.feature_log_prob_.T)
        ret += self.class_log_prior_.T
        return ret
    

In [254]:
l = LabelBinarizer()
Y = torch.cuda.FloatTensor(l.fit_transform(twenty_train.target)).cuda()


In [255]:
%%time
m = MultinomialNB()
m.fit(a, Y)

CPU times: user 26.2 ms, sys: 86 µs, total: 26.3 ms
Wall time: 10.5 ms


In [256]:
%%time
y_hat_gpu = m.predict(a)

CPU times: user 2.13 ms, sys: 4.55 ms, total: 6.67 ms
Wall time: 5.89 ms


In [265]:
y = twenty_train.target

In [266]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_hat_gpu.cpu().numpy())

0.9964554718653079

In [267]:
from sklearn.naive_bayes import MultinomialNB as MNB

In [268]:
%%time
sklearn_model = MNB()
sklearn_model.fit(X_train_counts, twenty_train.target)

CPU times: user 10.9 ms, sys: 632 µs, total: 11.5 ms
Wall time: 10.5 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [269]:
%%time
y_hat = sklearn_model.predict(X_train_counts)

CPU times: user 5.09 ms, sys: 0 ns, total: 5.09 ms
Wall time: 4.29 ms


In [270]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_hat)

0.9964554718653079

In [271]:
results = y - y_hat_gpu.cpu().numpy()
len(results[results!=0])

8